In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

with open(r'C:\Users\WMT\Downloads\anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))#set将文章中的所有不同字符取出，然后sorted排序
vocab_to_int = {c: i for i, c in enumerate(vocab)}#排好序的字符列表进行字典索引
int_to_vocab = dict(enumerate(vocab))#与上字典相反，索引号为键，字符为值
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)#把text中所有字符进行数字编码

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
print(len(text))

1985223


In [8]:
def get_batches(arr, n_seqs, n_steps):

    # 用sequence和step计算batch大小，得出batch个数，最后不够一个batch的扔掉
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    arr = arr[:n_batches * characters_per_batch]

    # 重新reshape为sequence行，列数自动生成（-1）
    arr = arr.reshape((n_seqs, -1))

    # 生成样本特征batch及目标值batch（目标值为样本值的下一个字母）
    for n in range(0, arr.shape[1], n_steps):
        x = arr[:, n:n+n_steps]
        y = np.zeros_like(x)
        # 目标值往下滚动一个字母，目标batch最后一列可设置为样本特征batch的第一列，不会影响精度
        y[:, :-1], y[:,-1] = x[:, 1:], x[:, 0]

        # x,y为生成器（generater）
        yield x, y

In [9]:
def build_inputs(batch_size, num_steps):
    '''batch_size是每个batch中sequence的长度（batch行数）
        num_steps是batch列数
    '''
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return inputs, targets, keep_prob


In [10]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):

    # 创建LSTM单元

    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)

        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop


    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)

    return cell, initial_state

In [11]:
def build_output(lstm_output, in_size, out_size):

    # reshape
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])

    # 将RNN输入连接到softmax层
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))

    logits = tf.matmul(x, softmax_w) + softmax_b
    out = tf.nn.softmax(logits, name='predictions')

    return out, logits

In [18]:
def build_loss(logits, targets, lstm_size, num_classes):

    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())

    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)

    return loss

In [24]:
def build_optimizer(loss, learning_rate, grad_clip):

    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))

    return optimizer

In [25]:
class CharRNN:

    def __init__(self, num_classes, batch_size=64, num_steps=50, lstm_size=128,
                num_layers=2, learning_rate=0.001, grad_clip=5, sampling=False):

        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()

        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        x_one_hot = tf.one_hot(self.inputs, num_classes)

        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state

        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)

        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [26]:
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers =2
learning_rate = 0.0001
keep_prob = 0.5

In [27]:
epochs = 20

# 每200步保存一个checkpoint
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps, 
               lstm_size=lstm_size, num_layers=num_layers, 
               learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)#  the maximum number of recent checkpoint files to keep.  As new files are created, older files are deleted.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    counter = 0
    for e in range(epochs):
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss,
                                                 model.final_state,
                                                 model.optimizer],
                                                 feed_dict=feed)
            end = time.time()
            print('Epoch:{}/{}...'.format(e+1, epochs),
                  'Training Step:{}...'.format(counter),
                  'Training loss:{:.4f}...'.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))

            if (counter % save_every_n == 0):
                saver.save(sess, 'checkpoints/i{}.ckpt'.format(counter))

    saver.save(sess, 'checkpoints/i{}.ckpt'.format(counter))

Epoch:1/20... Training Step:1... Training loss:4.4201... 5.2489 sec/batch
Epoch:1/20... Training Step:2... Training loss:4.4114... 0.3979 sec/batch
Epoch:1/20... Training Step:3... Training loss:4.4018... 0.3949 sec/batch
Epoch:1/20... Training Step:4... Training loss:4.3921... 0.4099 sec/batch
Epoch:1/20... Training Step:5... Training loss:4.3817... 0.3930 sec/batch
Epoch:1/20... Training Step:6... Training loss:4.3706... 0.4029 sec/batch
Epoch:1/20... Training Step:7... Training loss:4.3553... 0.4039 sec/batch
Epoch:1/20... Training Step:8... Training loss:4.3359... 0.3959 sec/batch
Epoch:1/20... Training Step:9... Training loss:4.3117... 0.3979 sec/batch
Epoch:1/20... Training Step:10... Training loss:4.2797... 0.3910 sec/batch
Epoch:1/20... Training Step:11... Training loss:4.2364... 0.3940 sec/batch
Epoch:1/20... Training Step:12... Training loss:4.1823... 0.3979 sec/batch
Epoch:1/20... Training Step:13... Training loss:4.0914... 0.3820 sec/batch
Epoch:1/20... Training Step:14... 

Epoch:1/20... Training Step:111... Training loss:3.2176... 0.3830 sec/batch
Epoch:1/20... Training Step:112... Training loss:3.2218... 0.4049 sec/batch
Epoch:1/20... Training Step:113... Training loss:3.2174... 0.4318 sec/batch
Epoch:1/20... Training Step:114... Training loss:3.2067... 0.3999 sec/batch
Epoch:1/20... Training Step:115... Training loss:3.2081... 0.4189 sec/batch
Epoch:1/20... Training Step:116... Training loss:3.2028... 0.4059 sec/batch
Epoch:1/20... Training Step:117... Training loss:3.2089... 0.3900 sec/batch
Epoch:1/20... Training Step:118... Training loss:3.2323... 0.3890 sec/batch
Epoch:1/20... Training Step:119... Training loss:3.2273... 0.3980 sec/batch
Epoch:1/20... Training Step:120... Training loss:3.1999... 0.4079 sec/batch
Epoch:1/20... Training Step:121... Training loss:3.2396... 0.4079 sec/batch
Epoch:1/20... Training Step:122... Training loss:3.2211... 0.3929 sec/batch
Epoch:1/20... Training Step:123... Training loss:3.2271... 0.4119 sec/batch
Epoch:1/20..

Epoch:2/20... Training Step:219... Training loss:3.1822... 0.3929 sec/batch
Epoch:2/20... Training Step:220... Training loss:3.1671... 0.3870 sec/batch
Epoch:2/20... Training Step:221... Training loss:3.1624... 0.3959 sec/batch
Epoch:2/20... Training Step:222... Training loss:3.1678... 0.3920 sec/batch
Epoch:2/20... Training Step:223... Training loss:3.1558... 0.3930 sec/batch
Epoch:2/20... Training Step:224... Training loss:3.1711... 0.3939 sec/batch
Epoch:2/20... Training Step:225... Training loss:3.1806... 0.3979 sec/batch
Epoch:2/20... Training Step:226... Training loss:3.1598... 0.3910 sec/batch
Epoch:2/20... Training Step:227... Training loss:3.1662... 0.3870 sec/batch
Epoch:2/20... Training Step:228... Training loss:3.1714... 0.3939 sec/batch
Epoch:2/20... Training Step:229... Training loss:3.1936... 0.3860 sec/batch
Epoch:2/20... Training Step:230... Training loss:3.1676... 0.3900 sec/batch
Epoch:2/20... Training Step:231... Training loss:3.1541... 0.3949 sec/batch
Epoch:2/20..

Epoch:2/20... Training Step:327... Training loss:3.1101... 0.3910 sec/batch
Epoch:2/20... Training Step:328... Training loss:3.1179... 0.3910 sec/batch
Epoch:2/20... Training Step:329... Training loss:3.1306... 0.3860 sec/batch
Epoch:2/20... Training Step:330... Training loss:3.1053... 0.3850 sec/batch
Epoch:2/20... Training Step:331... Training loss:3.1239... 0.3880 sec/batch
Epoch:2/20... Training Step:332... Training loss:3.1126... 0.3930 sec/batch
Epoch:2/20... Training Step:333... Training loss:3.0812... 0.4049 sec/batch
Epoch:2/20... Training Step:334... Training loss:3.0820... 0.3850 sec/batch
Epoch:2/20... Training Step:335... Training loss:3.0909... 0.3830 sec/batch
Epoch:2/20... Training Step:336... Training loss:3.0839... 0.3969 sec/batch
Epoch:2/20... Training Step:337... Training loss:3.1201... 0.3930 sec/batch
Epoch:2/20... Training Step:338... Training loss:3.1052... 0.3830 sec/batch
Epoch:2/20... Training Step:339... Training loss:3.0978... 0.3949 sec/batch
Epoch:2/20..

Epoch:3/20... Training Step:435... Training loss:2.9416... 0.4259 sec/batch
Epoch:3/20... Training Step:436... Training loss:2.9501... 0.3920 sec/batch
Epoch:3/20... Training Step:437... Training loss:2.9368... 0.4019 sec/batch
Epoch:3/20... Training Step:438... Training loss:2.9366... 0.3920 sec/batch
Epoch:3/20... Training Step:439... Training loss:2.9392... 0.3949 sec/batch
Epoch:3/20... Training Step:440... Training loss:2.9313... 0.3939 sec/batch
Epoch:3/20... Training Step:441... Training loss:2.9164... 0.3989 sec/batch
Epoch:3/20... Training Step:442... Training loss:2.9366... 0.3870 sec/batch
Epoch:3/20... Training Step:443... Training loss:2.9499... 0.4169 sec/batch
Epoch:3/20... Training Step:444... Training loss:2.9531... 0.3970 sec/batch
Epoch:3/20... Training Step:445... Training loss:2.9372... 0.3890 sec/batch
Epoch:3/20... Training Step:446... Training loss:2.9413... 0.3949 sec/batch
Epoch:3/20... Training Step:447... Training loss:2.9248... 0.4039 sec/batch
Epoch:3/20..

...

Epoch:18/20... Training Step:3526... Training loss:1.9609... 0.4269 sec/batch
Epoch:18/20... Training Step:3527... Training loss:1.9488... 0.4358 sec/batch
Epoch:18/20... Training Step:3528... Training loss:1.9188... 0.4418 sec/batch
Epoch:18/20... Training Step:3529... Training loss:1.9346... 0.4458 sec/batch
Epoch:18/20... Training Step:3530... Training loss:1.9288... 0.4289 sec/batch
Epoch:18/20... Training Step:3531... Training loss:1.9263... 0.4378 sec/batch
Epoch:18/20... Training Step:3532... Training loss:1.9128... 0.4398 sec/batch
Epoch:18/20... Training Step:3533... Training loss:1.9254... 0.4348 sec/batch
Epoch:18/20... Training Step:3534... Training loss:1.9629... 0.4368 sec/batch
Epoch:18/20... Training Step:3535... Training loss:1.9222... 0.4369 sec/batch
Epoch:18/20... Training Step:3536... Training loss:1.9186... 0.4428 sec/batch
Epoch:18/20... Training Step:3537... Training loss:1.9015... 0.4478 sec/batch
Epoch:18/20... Training Step:3538... Training loss:1.9085... 0.4

Epoch:19/20... Training Step:3632... Training loss:1.8741... 0.4318 sec/batch
Epoch:19/20... Training Step:3633... Training loss:1.8880... 0.4388 sec/batch
Epoch:19/20... Training Step:3634... Training loss:1.9000... 0.4329 sec/batch
Epoch:19/20... Training Step:3635... Training loss:1.9399... 0.4299 sec/batch
Epoch:19/20... Training Step:3636... Training loss:1.9114... 0.4398 sec/batch
Epoch:19/20... Training Step:3637... Training loss:1.9243... 0.4428 sec/batch
Epoch:19/20... Training Step:3638... Training loss:1.8907... 0.4508 sec/batch
Epoch:19/20... Training Step:3639... Training loss:1.8997... 0.4408 sec/batch
Epoch:19/20... Training Step:3640... Training loss:1.9422... 0.4488 sec/batch
Epoch:19/20... Training Step:3641... Training loss:1.9104... 0.4378 sec/batch
Epoch:19/20... Training Step:3642... Training loss:1.9030... 0.4369 sec/batch
Epoch:19/20... Training Step:3643... Training loss:1.8665... 0.4448 sec/batch
Epoch:19/20... Training Step:3644... Training loss:1.8887... 0.4

Epoch:19/20... Training Step:3738... Training loss:1.9003... 0.4368 sec/batch
Epoch:19/20... Training Step:3739... Training loss:1.9078... 0.4378 sec/batch
Epoch:19/20... Training Step:3740... Training loss:1.9100... 0.4468 sec/batch
Epoch:19/20... Training Step:3741... Training loss:1.8879... 0.4408 sec/batch
Epoch:19/20... Training Step:3742... Training loss:1.9171... 0.4418 sec/batch
Epoch:19/20... Training Step:3743... Training loss:1.8723... 0.4488 sec/batch
Epoch:19/20... Training Step:3744... Training loss:1.8727... 0.4528 sec/batch
Epoch:19/20... Training Step:3745... Training loss:1.8746... 0.4398 sec/batch
Epoch:19/20... Training Step:3746... Training loss:1.9040... 0.4518 sec/batch
Epoch:19/20... Training Step:3747... Training loss:1.9019... 0.4498 sec/batch
Epoch:19/20... Training Step:3748... Training loss:1.9230... 0.4359 sec/batch
Epoch:19/20... Training Step:3749... Training loss:1.9031... 0.4458 sec/batch
Epoch:19/20... Training Step:3750... Training loss:1.8765... 0.4

Epoch:20/20... Training Step:3844... Training loss:1.8940... 0.4378 sec/batch
Epoch:20/20... Training Step:3845... Training loss:1.8445... 0.4388 sec/batch
Epoch:20/20... Training Step:3846... Training loss:1.8706... 0.4319 sec/batch
Epoch:20/20... Training Step:3847... Training loss:1.8388... 0.4398 sec/batch
Epoch:20/20... Training Step:3848... Training loss:1.8625... 0.4388 sec/batch
Epoch:20/20... Training Step:3849... Training loss:1.8683... 0.4408 sec/batch
Epoch:20/20... Training Step:3850... Training loss:1.8454... 0.4468 sec/batch
Epoch:20/20... Training Step:3851... Training loss:1.8385... 0.4358 sec/batch
Epoch:20/20... Training Step:3852... Training loss:1.8884... 0.4348 sec/batch
Epoch:20/20... Training Step:3853... Training loss:1.8537... 0.4318 sec/batch
Epoch:20/20... Training Step:3854... Training loss:1.8718... 0.4358 sec/batch
Epoch:20/20... Training Step:3855... Training loss:1.8361... 0.4289 sec/batch
Epoch:20/20... Training Step:3856... Training loss:1.8648... 0.4

Epoch:20/20... Training Step:3950... Training loss:1.8547... 0.4488 sec/batch
Epoch:20/20... Training Step:3951... Training loss:1.8625... 0.4328 sec/batch
Epoch:20/20... Training Step:3952... Training loss:1.8632... 0.4428 sec/batch
Epoch:20/20... Training Step:3953... Training loss:1.8789... 0.4388 sec/batch
Epoch:20/20... Training Step:3954... Training loss:1.8453... 0.4348 sec/batch
Epoch:20/20... Training Step:3955... Training loss:1.8742... 0.4468 sec/batch
Epoch:20/20... Training Step:3956... Training loss:1.8431... 0.4378 sec/batch
Epoch:20/20... Training Step:3957... Training loss:1.8517... 0.4299 sec/batch
Epoch:20/20... Training Step:3958... Training loss:1.8744... 0.4338 sec/batch
Epoch:20/20... Training Step:3959... Training loss:1.8656... 0.4338 sec/batch
Epoch:20/20... Training Step:3960... Training loss:1.8703... 0.4388 sec/batch


In [28]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints\\i3960.ckpt"
all_model_checkpoint_paths: "checkpoints\\i200.ckpt"
all_model_checkpoint_paths: "checkpoints\\i400.ckpt"
all_model_checkpoint_paths: "checkpoints\\i600.ckpt"
all_model_checkpoint_paths: "checkpoints\\i800.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1000.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1200.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1400.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1600.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1800.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2000.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2200.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2400.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2600.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2800.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3000.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3200.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3400.ckpt"
all_model_checkpoint_paths: "checkpoi

In [29]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]]=0
    p = p / np.sum(p)
    char = np.random.choice(vocab_size, 1, p=p)[0]
    return char

In [30]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime='The '):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0, 0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction,
                                         model.final_state],
                                         feed_dict=feed)
            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])

            for i in range(n_samples):
                x[0,0] = c
                feed = {model.inputs: x,
                        model.keep_prob: 1.,
                        model.initial_state: new_state}
                preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

                c = pick_top_n(preds, len(vocab))
                samples.append(int_to_vocab[c])

    return ''.join(samples)         


In [31]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime='The')
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints\i3960.ckpt
Theelexd, "so dered in the read.

"That's mo be worling," thing the samaning of the ceating had the carre
tarking a thate hord foret all his wathed and to though, andwerind his
higenes to showed. They is as impentions were her the sand that to dean
hould to bece he his and at try wing, and shild neathing, and him not
she said to to the steress."

"Ahan I shale in the roomen theme't that?."

"Now, Ivin't anly to the seare of. I was santing that," she said a mint
thit wist were and ham bathir hes ant hid homese, and and what to shy
seeting and has seale an to bechines. But he her that has hissenfed the
sorter the soncesting, and the sungers it he cented thit hes sone and she
what had he did not so le to his woth and ant selfed and tomented, and the
would and all at one hin and himsenfe and with instechiog, and her hor
sampation with whome he hed soild to her she said, and and he hed a
tone a dowing. And he tanded herserf, h